In [ ]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry import mapping

import xarray as xr
from rasterio.io import MemoryFile
import numpy as np

nee_file = "../urban_greening/NEE.RS.FP-NONE.MLM-ALL.METEO-NONE.4320_2160.monthly.2015.nc"

In [ ]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import numpy as np

msa_file = '../urban_greening/msa/michiganMSA_reprojected.shp'
gpp_file = "../urban_greening/nov.15/michigan_test/modis-250-gpp-2015001.tif"

nlcd_file = "../urban_greening/nov.15/nlcd_2016_land_cover_l48_20210604.img"
ua_file = "../urban_greening/gis_processed/ua/ua_30.tif"


msa=gpd.read_file(msa_file)
msa=msa[msa['NAMELSAD']=='Grand Rapids-Wyoming-Kentwood, MI Metro Area'] # Subsetting to my AOI



with rasterio.open(gpp_file) as gpp_dstrd:
    msa_crsgpp=msa.to_crs(gpp_dstrd.crs)
    msa_crs = msa_crsgpp.crs
    gpp_crs = gpp_dstrd.crs
    gpp_clip_image, gpp_clip_transform=mask(gpp_dstrd,msa_crsgpp.geometry,crop=True)
    

    # replace fillvalue 65535 with np.nan
    gpp_clip_image = np.where(gpp_clip_image == 65535, np.nan, gpp_clip_image) 

    gpp_msa = gpp_clip_image[0]

    
    geometries = [mapping(geom) for geom in msa.geometry]

    with rasterio.open(nlcd_file) as nlcd_dstrd:
        nlcd_crs = nlcd_dstrd.crs
        
        msa_crsr = msa.to_crs(nlcd_crs)
        
        
        
        # Clip the raster using the reprojected geometries
        nlcd_clip_image, nlcd_clip_transform = mask(nlcd_dstrd, geometries, crop=True)
        
        nlcd_msa = nlcd_clip_image[0]

        

        import numpy as np
        gpp_msa_rr = np.empty((nlcd_clip_image.shape[1], nlcd_clip_image.shape[2]), dtype=np.float32) #make it support nan
        # gpp_msa_rr = np.empty((nlcd_clip_image.shape[1], nlcd_clip_image.shape[2]), dtype=gpp_dstrd.meta['dtype']), 
        from rasterio.enums import Resampling
        from rasterio.warp import calculate_default_transform, reproject
        reproject(
            source=gpp_msa,
            destination=gpp_msa_rr,
            src_transform=gpp_clip_transform,
            src_crs=gpp_crs,
            dst_transform=nlcd_clip_transform,
            dst_crs=nlcd_crs,
            resampling=Resampling.nearest, # use nearest to prevent unreasonable upscaling
            src_nodata=np.nan
        )
        
       

        with rasterio.open(ua_file) as ua_dstrd:
            ua_crs = ua_dstrd.crs

            ua_clip_image, ua_clip_transform = mask(ua_dstrd, geometries, crop=True)
            ua_msa = ua_clip_image[0]

            import numpy as np
            ua_msa_rr = np.empty((ua_clip_image.shape[1], ua_clip_image.shape[2]), dtype=ua_dstrd.meta['dtype'])

            from rasterio.enums import Resampling
            from rasterio.warp import calculate_default_transform, reproject
            reproject(
                source=ua_msa,
                destination=ua_msa_rr,
                src_transform=ua_clip_transform,
                src_crs=ua_crs,
                dst_transform=nlcd_clip_transform,
                dst_crs=nlcd_crs,
                resampling=Resampling.nearest
            )

        # nee_month_us_file = '../output/nee.tif'
        # with rasterio.open(nee_month_us_file) as nee_dstrd:
        with memfile.open() as nee_dstrd:
            nee_crs = gpp_crs
            
            # clip nee raster with shape
            nee_clip_image, nee_clip_transform = mask(nee_dstrd, [mapping(geom) for geom in msa_crsgpp.geometry], crop=True, all_touched=True)  # Include all touched pixels)
            nee_clip_image = np.where(nee_clip_image == -9999, np.nan, nee_clip_image) 
            
            nee_msa = nee_clip_image[0]

        memfile.close()
